# Part 3: Practical Data Preparation

**Objective:** Handle categorical features using One-Hot Encoding and address class imbalance using SMOTE.

## 1. Setup

Import necessary libraries.

In [64]:
import pandas as pd
import numpy as np
import os
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix
from sklearn.impute import SimpleImputer
from imblearn.over_sampling import SMOTE

## 2. Data Loading

Load the dataset.

In [65]:
def load_data(file_path):
    """
    Load the synthetic health data from a CSV file.
    
    Args:
        file_path: Path to the CSV file
        
    Returns:
        DataFrame containing the data
    """
    # YOUR CODE HERE
    # Load the CSV file using pandas

    df = pd.read_csv(file_path)
    
    return df


## 3. Categorical Feature Encoding

Implement `encode_categorical_features` using `OneHotEncoder`.

In [66]:
def encode_categorical_features(df, column_to_encode='smoker_status'):
    """
    Encode a categorical column using OneHotEncoder.
    
    Args:
        df: Input DataFrame
        column_to_encode: Name of the categorical column to encode
        
    Returns:
        DataFrame with the categorical column replaced by one-hot encoded columns
    """
    # YOUR CODE HERE
    # 1. Extract the categorical column
    # 2. Apply OneHotEncoder
    # 3. Create new column names
    # 4. Replace the original categorical column with the encoded columns

    categorical = df[[column_to_encode]]
    encoder = OneHotEncoder()
    enc_array = encoder.fit_transform(categorical)
    enc_df = pd.DataFrame(enc_array.toarray(), columns=encoder.get_feature_names_out())

    df_encoded = df.drop(columns=[column_to_encode])
    
    return pd.concat([df_encoded, enc_df], axis=1)

## 4. Data Preparation

Implement `prepare_data_part3` to handle the train/test split correctly.

In [67]:
def prepare_data_part3(df, test_size=0.2, random_state=42):
    """
    Prepare data with categorical encoding.
    
    Args:
        df: Input DataFrame
        test_size: Proportion of data for testing
        random_state: Random seed for reproducibility
        
    Returns:
        X_train, X_test, y_train, y_test
    """
    # YOUR CODE HERE
    # 1. Encode categorical features using the encode_categorical_features function
    # 2. Select relevant features (including the one-hot encoded ones) and the target
    # 3. Split data into training and testing sets
    # 4. Handle missing values
    
    cdf = encode_categorical_features(df)

    features_df = cdf[['age', 'systolic_bp', 'diastolic_bp', 'glucose_level', 'bmi','smoker_status_former', 'smoker_status_no', 'smoker_status_yes']]
    target = cdf['disease_outcome']

    imputer = SimpleImputer(strategy='mean') 
    imputer.fit(features_df)
    imputed_df = pd.DataFrame(imputer.transform(features_df), columns=features_df.columns)

    X_train, X_test, y_train, y_test = train_test_split(imputed_df, target, test_size=test_size, random_state=random_state)

    return X_train, X_test, y_train, y_test

## 5. Handling Imbalanced Data

Implement `apply_smote` to oversample the minority class.

In [68]:
def apply_smote(X_train, y_train, random_state=42):
    """
    Apply SMOTE to oversample the minority class.
    
    Args:
        X_train: Training features
        y_train: Training target
        random_state: Random seed for reproducibility
        
    Returns:
        Resampled X_train and y_train with balanced classes
    """
    # YOUR CODE HERE
    # Apply SMOTE to balance the classes
    
    smote= SMOTE(random_state=random_state)
    X_re_train, y_re_train = smote.fit_resample(X_train, y_train)

    return X_re_train, y_re_train

## 6. Model Training and Evaluation

Train a model on the SMOTE-resampled data and evaluate it.

In [69]:
def train_logistic_regression(X_train, y_train):
    """
    Train a logistic regression model.
    
    Args:
        X_train: Training features
        y_train: Training target
        
    Returns:
        Trained logistic regression model
    """
    # YOUR CODE HERE
    # Initialize and train a LogisticRegression model

    lrmodel = LogisticRegression()
    lrmodel.fit(X_train, y_train)
    
    return lrmodel  

In [70]:
def calculate_evaluation_metrics(model, X_test, y_test):
    """
    Calculate classification evaluation metrics.
    
    Args:
        model: Trained model
        X_test: Test features
        y_test: Test target
        
    Returns:
        Dictionary containing accuracy, precision, recall, f1, auc, and confusion_matrix
    """
    # YOUR CODE HERE
    # 1. Generate predictions
    # 2. Calculate metrics: accuracy, precision, recall, f1, auc
    # 3. Create confusion matrix
    # 4. Return metrics in a dictionary

    y_pred = model.predict(X_test)
    
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    auc = roc_auc_score(y_test, y_pred)

    conf_matrix = confusion_matrix(y_test, y_pred)
    
    return {'accuracy':accuracy, 'precision':precision, 'recall':recall, 'f1':f1, 'auc':auc, 'confusion_matrix':conf_matrix}


## 7. Save Results

Save the evaluation metrics to a text file.

In [71]:
# YOUR CODE HERE
# 1. Create 'results' directory if it doesn't exist
# 2. Format metrics as strings
# 3. Write metrics to 'results/results_part3.txt'

#with open('results/results_part3.txt', 'w') as file:
    #for name, value in metric.items():
            #file.write(f"{name}: {value} \n")

## 9. Compare Results

Implement a function to compare model performance between balanced and imbalanced data.

In [72]:
def compare_models(part1_metrics, part3_metrics):
    """
    Calculate percentage improvement between models trained on imbalanced vs. balanced data.
    
    Args:
        part1_metrics: Dictionary containing evaluation metrics from Part 1 (imbalanced)
        part3_metrics: Dictionary containing evaluation metrics from Part 3 (balanced)
        
    Returns:
        Dictionary with metric names as keys and improvement percentages as values
    """
    # YOUR CODE HERE
    # 1. Calculate percentage improvement for each metric
    # 2. Handle metrics where higher is better (most metrics) and where lower is better
    # 3. Return a dictionary with metric names and improvement percentages
    
    percentage = {}
    
    for metric in part3_metrics:
        if metric != 'confusion_matrix':  # Exclude confusion_matrix 
            percentage[metric] = 100 * ( (part3_metrics[metric] / part1_metrics[metric]) -1)
            #100 * (part3_metrics[metric] - part1_metrics[metric]) / part3_metrics[metric] um not percents more like 0.something
    return percentage 

## 8. Main Execution

Run the complete workflow.

In [73]:
# Main execution
if __name__ == "__main__":
    # 1. Load data
    data_file = 'data/synthetic_health_data.csv'
    df = load_data(data_file)
    
    # 2. Prepare data with categorical encoding
    X_train, X_test, y_train, y_test = prepare_data_part3(df)
    
    # 3. Apply SMOTE to balance the training data
    X_train_resampled, y_train_resampled = apply_smote(X_train, y_train)
    
    # 4. Train model on resampled data
    model = train_logistic_regression(X_train_resampled, y_train_resampled)
    
    # 5. Evaluate on original test set
    metrics = calculate_evaluation_metrics(model, X_test, y_test)
    
    # 6. Print metrics
    for metric, value in metrics.items():
        if metric != 'confusion_matrix':
            print(f"{metric}: {value:.4f}")
    
    # 7. Save results
    with open('results/results_part3.txt', 'w') as file:
        for name, value in metrics.items():
            file.write(f"{name}: {value} \n")
    
    # 8. Load Part 1 results for comparison
    import json
    try:
        with open('results/results_part1.txt', 'r') as f:
            part1_metrics = json.load(f)
        
        # 9. Compare models
        comparison = compare_models(part1_metrics, metrics)
        print("\nModel Comparison (improvement percentages):")
        for metric, improvement in comparison.items():
            print(f"{metric}: {improvement:.2f}%")
    except FileNotFoundError:
        print("Part 1 results not found. Run part1_introduction.ipynb first.")

accuracy: 0.8104
precision: 0.3200
recall: 0.8392
f1: 0.4633
auc: 0.8232

Model Comparison (improvement percentages):
accuracy: -11.61%
precision: -51.63%
recall: 179.07%
f1: 12.06%
auc: 28.22%


/home/codespace/.local/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
